In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import json
import seaborn as sns
from random import randint


In [ ]:
# from py_spotipy_data_wrangling import streams, new_features, get_features, local_features, streams_af_df, no_skip_df, streams_af_df, cols_to_merge, sp
# todo: remove unused variables
streams_af_df = pd.read_pickle("./streams_af_df.p")
streams_df = pd.read_pickle("./streams_df.p")
no_skip_df = pd.read_pickle("./no_skip_df.p")
playlist_af_df = pd.read_pickle("./playlist_af_df.p")

cid = "76c42909213b45a4a22596b35328705f"
secret = "9229ae6311b54925ad2f62ad4e32ccf6"
username = "nshsunmalb8t3brxxz0ks3e47"
sp = spotipy.Spotify(
    client_credentials_manager=SpotifyClientCredentials(
        client_id=cid, client_secret=secret
    )
)


In [ ]:
# series2 = streams_af_df['spotify_track_uri']
# series1 = streams['spotify_track_uri']
# comparison = series1[~series1.isin(series2)]
# comparison


In [ ]:
# sp.audio_features('77cVT85pU7WH3elC1SE7Uu')


In [ ]:
# streams.query('spotify_track_uri in @comparison')


In [ ]:
# missing_tracks = []
# for i in range(len(comparison)):
#     missing_tracks.append(sp.track(comparison.iloc[i])['external_urls'])
# missing_tracks


In [ ]:
streams_af_df


In [ ]:
streams_af_df[streams_af_df.ts.astype(
    str).duplicated() == True].sort_values(by='ts')

In [ ]:
# I don't recall listening to this artist so much. Investigating if it was on repeat on accident or something.

why_tala = streams_af_df.query('"Hope Tala" in artist')
why_why_tala = why_tala.groupby(by='date').sum(numeric_only=True
).sort_values(by='ms_played', ascending=False)
why_why_tala['ms_played'] = why_why_tala['ms_played'].apply(
    lambda x: x / 60000)
why_why_tala.head(50)

In [ ]:
# Looks like I just let her run on most popular song repeat for a few hours.
streams_af_df.query('date == "11/27/2019"').sort_values(by="ts")


In [ ]:
no_skip_df.query('artist.str.contains("(?i)miles davis")', engine='python')


In [ ]:
pivot_track = pd.DataFrame(no_skip_df.pivot_table(
    columns=['artist', 'track'], aggfunc='size').sort_values(ascending=False))
pivot_track

In [ ]:
# # Check for tracks missing from new df
# series1 = duration_prep['spotify_track_uri']
# series2 = local_features['uri']
# comparison = series1[~series1.isin(series2)]
# comparison

In [ ]:
# # Show missing track URLs for further investigation; track is likely removed or yet to be analyzed for audio_features.
# missing_tracks = []
# for i in range(len(comparison)):
#     missing_tracks.append(sp.track(comparison.iloc[i])['external_urls'])
# missing_tracks


In [ ]:
# Most played tracks
pivot_track = pd.DataFrame(no_skip_df.pivot_table(
    columns=['artist', 'track'], aggfunc='size').sort_values(ascending=False))
pivot_track

In [ ]:
# Most listened to artists by number of songs played
pivot_artist = pd.DataFrame(no_skip_df.pivot_table(
    columns=['artist'], aggfunc='size').sort_values(ascending=False))
pivot_artist

In [ ]:
no_skip_df


In [ ]:
miles = no_skip_df.query('artist.str.contains("(?i)Miles Davis")')
pivot_miles = pd.pivot_table(miles, values='ms_played', index=[
                             'album'], aggfunc='sum')
pivot_miles = pivot_miles.sort_values(by='ms_played', ascending=False)
miles

In [ ]:
pivot_album_time = pd.pivot_table(
    no_skip_df, values='ms_played', index=['album'], aggfunc='sum')

In [ ]:
pivot_album_time = pivot_album_time.sort_values(
    by='ms_played', ascending=False)
pivot_album_time.head(20)

In [ ]:
pivot_artist_time = pd.pivot_table(
    no_skip_df, values="ms_played", index=["artist"], aggfunc="sum"
)


In [ ]:
# print(pivot_artist_time.sort_values().to_string())
artist_time = pivot_artist_time.sort_values(
    by='ms_played', ascending=False).reset_index()
print(artist_time.to_string())

In [ ]:
# Artists played more than 10 mins
repeated_artists = artist_time[:1750]
# Number of unique tracks played from those artists

len(streams_af_df["track"].unique())


In [ ]:
still_repeated = repeated_artists["artist"]


In [ ]:
# Count number of unique tracks per artist and create ascending list
still_still_repeated = no_skip_df.query("artist in @still_repeated")
# still_still_repeated.loc[:, 'artist'] = still_still_repeated.copy().loc[:, 'artist'].astype(str)
group = still_still_repeated.groupby("artist")
songs_per_artist = group.apply(lambda x: x["track"].unique())
unique_songs = songs_per_artist.reset_index(name="unique_titles")


In [ ]:
unique_songs['unique_count'] = unique_songs.iloc[:, 1].str.len()

In [ ]:
unique_songs = unique_songs.sort_values(by="unique_count").reset_index()


In [ ]:
unique_export = unique_songs.query("unique_count <= 8")["artist"]


In [ ]:
pivot_artist_time.ms_played


In [ ]:
# Artists with 10 or less songs played that I've heard more than an hour of. Indicative of artists I should explore more of.
pivot_artist_time.query('ms_played > 3600000').query(
    'artist in @unique_export')

Todo: Next step after the above is to plot least uniques with most ms played

In [ ]:
# Convert
pivot_artist_time['ms_played'] = pd.to_timedelta(
    pivot_artist_time.ms_played, unit='ms')

In [ ]:
def format_td(td):
    '''Converts timedelta to hh:mm:ss strictly for presentation purposes'''
    minutes, seconds = divmod(td.seconds + td.days * 86400, 60)
    hours, minutes = divmod(minutes, 60)
    return '{:d}:{:02d}:{:02d}'.format(hours, minutes, seconds, td.microseconds)

In [ ]:
pivot_artist_time['ms_played'] = pivot_artist_time['ms_played'].apply(format_td)

In [ ]:
# Show longest songs
no_skip_df.sort_values(by="ms_played", axis=0, ascending=False)


In [ ]:
# How to control this for missing months? do october 25 2019 thru 2022
sns_prep = streams.query('year == "2019"| year == "2020" | year == "2021"')
sns_prep = pd.pivot_table(sns_prep, values='ms_played', index=[
                          'month'], aggfunc='sum',).reset_index()
sns_prep['hours played'] = sns_prep['ms_played'].apply(lambda x: x / 3600000)
sns.barplot(data=sns_prep, x='month', y='hours played')

In [ ]:
# sum of unique uris closely maps to total skips per month
sns_prep = streams_af_df.copy().drop_duplicates(subset='id')
sns_prep['count'] = 1
sns_prep = sns_prep.loc[:, ['month', 'duration_ms', 'ms_played',
                            'shuffle', 'count']].groupby(by='month').sum(numeric_only=True)
sns.barplot(data=sns_prep, x=sns_prep.index, y='count')

In [ ]:
# good way to find months with most music discovery along with unique URIs
sns_prep = streams_af_df.copy().query('(ms_played / duration_ms) < 0.25')
sns_prep.at[:, 'count'] = 1
sns_prep = sns_prep.loc[:, ['month', 'duration_ms', 'ms_played',
                            'shuffle', 'count']].groupby(by='month').sum(numeric_only=True)
sns.barplot(data=sns_prep, x=sns_prep.index, y='count')

In [ ]:
# common keys of listened tracks
no_skip_df.loc[:, 'ts'] = no_skip_df.loc[:, 'ts'].astype(str)
pivot_track = pd.DataFrame(no_skip_df.drop_duplicates(subset='ts').pivot_table(
    columns=['key_signature'], aggfunc='size').sort_values(ascending=False))
pivot_track

In [ ]:
def open_wheel():
    with open("camelot.json") as json_file:
        camelot_json = json.load(json_file)
        wheel = camelot_json
        return wheel

In [ ]:
# Todo: see if df approach is better than dict approach
# compute is about the same, probably turn this approach into a function
wheel = open_wheel()
wheel = pd.DataFrame.from_dict(wheel)
song_key = playlist_af_df.loc[1, 'camelot']
friendkey = wheel.loc['dominant_relative', song_key]
playlist_af_df.query('camelot in @friendkey')

In [ ]:
# # This version works with uri
# #should also have function to get uri from song title + artist
# #todo: proper type hinting and default values
# # separate functions i suppose, maybe with decorators
# # https://stackoverflow.com/questions/62153371/best-way-to-create-python-function-with-multiple-options


def get_friendly(
    df,
    tempo_range=10,
    uri=None,
    index=None,
    shuffle=None,
    playlist=False,
    shift=["all"],
):
    wheel = open_wheel()
    df = df.drop_duplicates(subset="id").reset_index()
    if uri:
        song_selected = df.loc[df["id"] == uri].iloc[0]
    elif index or index == 0:
        song_selected = df.loc[index]
    elif shuffle:
        song_selected = df.iloc[randint(0, len(df) - 1)]
    else:
        print(
            "Error: no song selected. Specify shuffle=True to operate on random song."
        )
    # Designate desired tempo range
    selected_tempo = song_selected["tempo"]
    acceptable_tempos = range(
        (selected_tempo - tempo_range), (selected_tempo + tempo_range), 1
    )

    # Select harmonically compatible key signatures in camelot.json
    friendly_keys = []
    for i in range(len(shift)):
        key = wheel[song_selected["camelot"]][shift[i]]
        friendly_keys.append(key)
        print(key)
        if type(key) == list:
            friendly_keys.extend(key)
    # Show tracks with harmonically compatible key signatures within a given tempo range. Accounts for Spotify's tendency to double or halve numeric tempos.
    return df.query(
        "camelot in @friendly_keys & (tempo in @acceptable_tempos | tempo * 2 in @acceptable_tempos | tempo / 2 in @acceptable_tempos)"
    )


# get_friendly(playlist_af_df, tempo_range=10, uri='5jh3f8fy8a335XYDyJ7x9R', shift=['all']).sort_values(by='track')
get_friendly(streams_af_df, tempo_range=10, index=0, shift=["all"]).sort_values(
    by="track"
)
# get_friendly(playlist_af_df, tempo_range=10, shuffle=True, shift=['all']).sort_values(by='track')


In [ ]:
streams_af_df.loc[0]
